In [3]:
#Jupyter_DASH
from jupyter_dash import JupyterDash
import numpy as np
import pandas as pd
import dash
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import dash_html_components as html
from dash.dependencies import Input, Output, State
import math

app = JupyterDash(__name__, external_stylesheets=[dbc.themes.LUX])

#se cargar el archivo
df = pd.read_excel('cancer_mama.xlsx', squeeze=True)

#se redondea la columnna edad de diagnostico
df['age_at_diagnosis'] = np.round(df['age_at_diagnosis'])

#se eliminan columnas con muy poca informacion
df = df.drop(df.columns[range(520, 693)], axis=1)   

# Define the dropdown values for the multi axis graph
multiAxisColumnNames = [
    {'value': 'age_at_diagnosis', 'label': 'Age at Diagnosis'},
    {'value': 'type_of_breast_surgery', 'label': 'Type of Breast Surgery'},
    {'value': 'cancer_type_detailed', 'label': 'Type of Breast Cancer'},
    {'value': 'chemotherapy', 'label': 'Chemotherapy'},
    {'value': 'overall_survival', 'label': 'Survival'},
]
cancer_type_detailed = [cancer for cancer in df['cancer_type_detailed'].unique()]


#----------------------------------------------------------------------------------------------------------------------
# Este es el Navbar, es la barra que se muestra en el tope de la página
navbar = dbc.Navbar([
    html.A(
        dbc.Row(
            [
                dbc.Col(html.Img(src=app.get_asset_url("logo.png"), height="40px"), width=3),
                dbc.Col(dbc.NavbarBrand("OncoView"), width=9, align="center"),
            ],
            align="center",
            no_gutters=True,
        ),
    )
],
    color="lightblue"
)


# ------------------------------------------------------------------------------------------------------------------------------

# Este es el layout general de la página, Aquí va el gráfico de cambio de eje

layout_general = html.Div([
    html.Br(),
    html.Br(),
    html.H4("Factors Involved in Breast Cancer Patient Outcome"),
    html.Br(),
    html.Br(),
    dbc.Row([
            dbc.Col(html.Div([
                dbc.Card(
                    dbc.CardBody(
                        [
                            # Aqui se muestran los filtros que afectan todo el dataset
                            # La idea de estos filtros es usarlos en todos los callbacks
                            # Y aplicarlos a las copias de los datasets en cada callback
                            html.H5("Filter Panel"),
                            html.Br(),
                            html.Div([
                                
                                # Filtro de edad, usamos un rango mayor al que tenemos en realidad
                                # (tenemos de 22 a 96 años) para que sea más fácil representarlo visualmente
                                # en el UI. El widget para este filtro es un range slider que va de 20 a 100
                                # en incrementos de 10 en 10 para hacerlo más fácil de visualizar
                                html.H5('Age at Diagnosis'), 
                                dcc.RangeSlider(
                                    id="age_filter",
                                    marks={i: str(i) for i in range(20, 101, 10)},
                                    min=20,
                                    max=100,
                                    value=[22, 96]
                                ),
                                
                                # Filtro de quimioterapia, es una lista de checkbox que tiene dos opciones
                                # Pacientes con quimioterapia
                                # Pacientes sin quimioterapia
                                html.Br(),
                                html.H5('Chemotherapy'), 
                                dcc.Checklist(
                                    id="chemotherapy_filter",
                                    options=[
                                        {'label': ' Yes', 'value': 1},
                                        {'label': ' No', 'value': 0}
                                    ],
                                    value=[0, 1],
                                    labelStyle = dict(display='block')
                                ),
                                
                                # Filtro de supervivencia, es una lista de checkbox que tiene dos opciones
                                # Pacientes que sobrevivieron
                                # Pacientes que no sobrevivieron
                                html.Br(),
                                html.H5('Survival'), 
                                dcc.Checklist(
                                    id="survival_filter",
                                    options=[
                                        {'label': ' Yes', 'value': 1},
                                        {'label': ' No', 'value': 0}
                                    ],
                                    value=[0, 1],
                                    labelStyle = dict(display='block')
                                ),
                                
                                # Filtro de tipo de cancer. Aqui las opciones se crean usando
                                # los valores de la columna en el dataset. Hacemos un query que
                                # retorna los valores unicos de la columna cancer_type_detailed
                                # y eso es lo que ponemos como valores en el dropdown para este filtro
                                html.Br(),
                                html.H5('Select Cancer Type'), 
                                # Dropdown en value tenemos el valor por default  
                                dcc.Dropdown(
                                    id="cancer_type_filter",
                                    options=[{'label': cancer, 'value': cancer} for cancer in cancer_type_detailed],
                                    value=cancer_type_detailed,
                                    multi=True,
                                    placeholder="Select Cancer Type"
                                ) 

                            ]),
                        ]
                    )
                )  
            ]), width=3),
            dbc.Col(html.Div([
                # Este filtro es el dropdown para seleccionar el eje x que se quiere visualizar en
                # el gráfico de multi-axis. Los valores que se le ponen al dropdown estan definidos arriba
                html.H5('Select an Axis to Visualize the Number of Patients'), 
                html.Br(),
                dcc.Dropdown(
                    id="multi_axis_selection",
                    options=multiAxisColumnNames,
                    multi=False,
                    value="age_at_diagnosis",
                    style={"width": "75%"},
                    placeholder="Select an Option"
                ),
                html.Br(),
                
                # Aqui definimos el gráfico de multi-axis
                html.H5(id='multi_axis_label', children=['Multi Axis Graph']), 
                html.Br(),
                dcc.Graph(id='multi_axis_graph')                    
            ]), width=9),
        ]
    ),

], style={"marginLeft": "10px", "marginRight": "10px"})



# -----------------------------------------------------------------------------------------------------------------------------

    
# Este es el callback que usamos para crear el gráfico de multiaxis
# Tenemos varios inputs que son los valores seleccionados en los diferentes filtros por el usuario
# El output es el gráfico de multi-axis que habíamos definido
@app.callback(
    [
        Output(component_id='multi_axis_graph', component_property='figure'),
        Output(component_id='multi_axis_label', component_property='children')
    ],
    [
        Input(component_id='age_filter', component_property='value'),
        Input(component_id='chemotherapy_filter', component_property='value'),
        Input(component_id='survival_filter', component_property='value'),
        Input(component_id='cancer_type_filter', component_property='value'),
        Input(component_id='multi_axis_selection', component_property='value')
    ],
    State("multi_axis_selection","options")
)
def update_graph(
    age_range_filter, 
    chemotherapy_filter, 
    survival_filter, 
    cancer_type_filter, 
    column_selected,
    column_options
):
    graph_title = "Multiple Axis Graph"
    column_selected_label = ""
    for option in column_options:
        if option['value'] == column_selected:
            column_selected_label = option['label']
            graph_title = "Count of patients per {}".format(column_selected_label)
    
    # Hacemos una copia del dataset
    dff = df.copy()
    # Aplicamos los filtros generales
    dff = apply_age_range_filter(dff, age_range_filter)
    dff = apply_chemotherapy_filter(dff, chemotherapy_filter)
    dff = apply_survival_filter(dff, survival_filter)
    dff = apply_cancer_type_filter(dff, cancer_type_filter)
    
    # Creamos la columna count en el dataset
    dff['count']=1
    # Hacemos un groupby y un sum para saber la cantidad de pacientes por categoría
    # Aquí el column_selected es la columna que seleccionó el usuario para mostrar en el gráfico de multiindex
    dff = dff[[column_selected, 'count']].groupby([column_selected]).sum().reset_index()
    # Esto es una validación en caso de que el dataset llegue vació por culpa de los filtros
    # Ocupamos crear un mensaje de error o algo similar para mostrar al usuario
    if dff.empty:
        return {}, graph_title
    return px.bar(
        dff,
        x=column_selected,
        y="count",
        color=column_selected,
        labels={
             column_selected: column_selected_label,
             "count": "Number of Patients"
         }
    ), graph_title


# Función para aplicar el filtro por edad
def apply_age_range_filter(dataset, age_range):
    if age_range is not None:
        return dataset[dataset['age_at_diagnosis'].isin(range(age_range[0], age_range[1] + 1))]
    return dataset
    
    
# Función para aplicar el filtro por quimioterapia
def apply_chemotherapy_filter(dataset, chemotherapy):
    if chemotherapy is not None:
        temp_dataset = dataset[dataset['chemotherapy'].isin(chemotherapy)]
        temp_dataset.loc[temp_dataset['chemotherapy'] == 0, 'chemotherapy'] = "No"
        temp_dataset.loc[temp_dataset['chemotherapy'] == 1, 'chemotherapy'] = "Yes"
        return temp_dataset
    return dataset


# Función para aplicar el filtro por supervivencia
def apply_survival_filter(dataset, survival):
    if survival is not None:
        temp_dataset = dataset[dataset['overall_survival'].isin(survival)]
        temp_dataset.loc[temp_dataset['overall_survival'] == 0, 'overall_survival'] = "No"
        temp_dataset.loc[temp_dataset['overall_survival'] == 1, 'overall_survival'] = "Yes"
        return temp_dataset
    return dataset
    

# Función para aplicar el filtro por tipo de cancer
def apply_cancer_type_filter(dataset, cancer_type):
    if cancer_type is not None:
        return dataset[dataset['cancer_type_detailed'].isin(cancer_type)]
    return dataset


#--------------------------------------------------------------------------------------------------------------------------------

# Aquí va el layout de la página con gráficos específicos (más elaborados)
layout_especifico = html.Div([
    html.Br(),
    html.Br(),
    html.H4("Factors Involved in Breast Cancer Patient Outcome"),
    html.Br(),
    html.Br(),
    dbc.Row(
            [
                dbc.Col(html.Div([
                    dbc.Card(
                    dbc.CardBody(
                        [
                    html.H5('Filter Panel'),
                    html.Br(),
                        html.H5('Age at Diagnosis'), 
                    dcc.RangeSlider(id="agerange_filter",
                                    marks={i: str(i) for i in range(20, 101, 10)},
                                    min=20,
                                    max=100,
                                    value=[22, 96]
                                ),
                            
                    html.Br(),
                        html.H5('Chemotherapy'), 
                    dcc.Checklist(id="chemotherapy_filterYN",
                                  options=[
                                        {'label': ' Yes', 'value': 1},
                                        {'label': ' No', 'value': 0}
                                    ],
                                    value=[0, 1],
                                    labelStyle = dict(display='block')
                                ),
                   
                    html.Br(),
                        html.H5('Radiotherapy'), 
                    dcc.Checklist(id="Radiotherapy_filterYN",
                                  options=[
                                        {'label': ' Yes', 'value': 1},
                                        {'label': ' No', 'value': 0}
                                    ],
                                    value=[0, 1],
                                    labelStyle = dict(display='block')
                                ),         
                            
                    html.Br(),
                        html.H5('Hormone Therapy'), 
                    dcc.Checklist(id="Hormonetherapy_filterYN",
                                  options=[
                                        {'label': ' Yes', 'value': 1},
                                        {'label': ' No', 'value': 0}
                                    ],
                                    value=[0, 1],
                                    labelStyle = dict(display='block')
                                ),         
                                    
                    html.Br(),
                        html.H5('Survival'), 
                    dcc.Checklist(id="survival_filterYN",
                                  options=[
                                       {'label': ' Yes', 'value': 1},
                                       {'label': ' No', 'value': 0}
                                    ],
                                    value=[0, 1],
                                  labelStyle = dict(display='block')
                                ),
    
                    html.Br(),
                            html.H5('Select Cancer Type'), 
                    dcc.Dropdown (id="cancer_type",
                                  options=[
                                      {'label': i, 'value': i} for i in df.cancer_type_detailed.unique()],
                                  value=cancer_type_detailed,
                                  multi=True, 
                                  placeholder="Select Cancer Type"
                                 ),
                    
                ]))]), width=3),
                
                dbc.Col(html.Div([
                    
                    html.Br(),
                    
                    html.H5('Overall Survival of Patients over Age at Diagnosis per Neoplasm Histologic Grade'), 
                    dcc.Graph(id='scatterplot', figure={}),
                    
                     html.Br(),
                    
                    html.H5('Expression of Oncogenes and Tumor Suppresor Genes over Cancer Type'),
                    html.Br(),
                    dcc.Graph(id='htmp', figure={}),
                    
                ])),
            ]
        )

], style={"marginLeft": "10px", "marginRight": "10px"})




# -----------------------------------------------------------------------------------------------------------------------------    
#callback especificos


@app.callback([Output(component_id='scatterplot', component_property='figure'),
               Output(component_id='htmp', component_property='figure')],
              [Input(component_id='agerange_filter', component_property='value'),
               Input(component_id='chemotherapy_filterYN', component_property='value'),
               Input(component_id='Radiotherapy_filterYN', component_property='value'),
               Input(component_id='Hormonetherapy_filterYN', component_property='value'),
               Input(component_id='survival_filterYN', component_property='value'),
               Input(component_id='cancer_type', component_property='value')]
)


def update_graph(agerange_filter,
                 chemotherapy_filterYN,
                 Radiotherapy_filterYN,
                 Hormonetherapy_filterYN,
                 survival_filterYN,
                 cancer_type):
    
    dff = df.copy()
    dff1 = df.copy()
    # Aplicamos los filtros generales
    dff = apply_agerange_filter(dff, agerange_filter)
    dff = apply_chemotherapy_filterYN(dff, chemotherapy_filterYN)
    dff = apply_Radiotherapy_filterYN(dff, Radiotherapy_filterYN)
    dff = apply_Hormonetherapy_filterYN(dff, Hormonetherapy_filterYN)
    dff = apply_survival_filterYN(dff, survival_filterYN)
    dff = apply_cancer_type(dff, cancer_type)
    dff1 = apply_agerange_filter(dff1, agerange_filter)
    #dff1 = apply_survival_filterYN(dff1, survival_filterYN)
    dff1 = apply_cancer_type(dff1, cancer_type)

    #Scatter plot sin el overlap y con el color by de grado de neoplasia y el symbol de celularidad
    dff['Overall_survival_in_years']=(dff['overall_survival_months']/12)
    dff.chemotherapy.loc[dff.chemotherapy == 1] = 'Yes'
    dff.chemotherapy.loc[dff.chemotherapy == 0] = 'No'
    dff.hormone_therapy.loc[dff.hormone_therapy == 1] = 'Yes'
    dff.hormone_therapy.loc[dff.hormone_therapy == 0] = 'No'
    dff.radio_therapy.loc[dff.radio_therapy == 1] = 'Yes'
    dff.radio_therapy.loc[dff.radio_therapy == 0] = 'No'
    dff['neoplasm_histologic_grade']=dff['neoplasm_histologic_grade'].astype('str')
    fig = px.scatter(dff, x='Overall_survival_in_years', y="age_at_diagnosis", color='neoplasm_histologic_grade', 
                 labels={'Overall_survival_in_years':'Overall Survival (Years)', "age_at_diagnosis":'Age at Diagnosis',
                        'neoplasm_histologic_grade': 'Neoplasm Histologic Grade', 'chemotherapy': 'Chemotherapy',
                        'hormone_therapy': 'Hormone Therapy', 'type_of_breast_surgery': 'Type of Breast Surgery', 
                         'radio_therapy':'Radio Therapy', "cellularity": "Cellularity"}, 
                 hover_data=['chemotherapy', 'hormone_therapy', 'type_of_breast_surgery', 'radio_therapy'],
                 color_continuous_scale="sunset",
                 #title = "Overall Survival of Patients over Age at Diagnosis per Neoplasm Histologic Grade",  
                 symbol = "cellularity") 
    
    
    #Heatmap
    #dff1 = df.copy()
    dfA1=dff1[['cancer_type_detailed','myc','hras','kras','nras','brca1','brca2','chek2','pten','pik3ca','akt1','akt2']]
    dfA1_unpivoted = dfA1.melt(id_vars=['cancer_type_detailed'], var_name='Oncogenes', value_name='Results')
    #print(dfA1_unpivoted)
    dfB1 = dfA1_unpivoted.groupby(['cancer_type_detailed','Oncogenes'])[['Results']].mean().reset_index()
    #print(dfB1)
    dfC1 = dfB1.pivot(index= 'cancer_type_detailed', columns = 'Oncogenes', values='Results')


    heatmpgraph = px.imshow(dfC1, labels=dict(x="Oncogenes & Tumor Suppressor Genes", y="Cancer Type Detailed", color="Expression Levels"), 
                            color_continuous_scale="RdBu")
    heatmpgraph.update_xaxes(side="top")

    
    #Se debe de retornar la cantidad de outputs que se tengan
    return fig, heatmpgraph
    

# Función para aplicar el filtro por edad
def apply_agerange_filter(dataset, age_slctd):
    if bool(age_slctd):
        return dataset[dataset['age_at_diagnosis'].isin(range(age_slctd[0], age_slctd[1] + 1))]
    return dataset

# Función para aplicar el filtro por quimioterapia
def apply_chemotherapy_filterYN(dataset, chemotherapy_slctd):
    if bool(chemotherapy_slctd):
        temp_dataset = dataset[dataset['chemotherapy'].isin(chemotherapy_slctd)]
        temp_dataset.loc[temp_dataset['chemotherapy'] == 0, 'chemotherapy'] = "No"
        temp_dataset.loc[temp_dataset['chemotherapy'] == 1, 'chemotherapy'] = "Yes"
        return temp_dataset
    return dataset

# Función para aplicar el filtro por quimioterapia
def apply_Radiotherapy_filterYN(dataset, radio_therapy_slctd):
    if bool(radio_therapy_slctd):
        temp_dataset = dataset[dataset['radio_therapy'].isin(radio_therapy_slctd)]
        temp_dataset.loc[temp_dataset['radio_therapy'] == 0, 'radio_therapy'] = "No"
        temp_dataset.loc[temp_dataset['radio_therapy'] == 1, 'radio_therapy'] = "Yes"
        return temp_dataset
    return dataset

# Función para aplicar el filtro por quimioterapia
def apply_Hormonetherapy_filterYN(dataset, hormone_therapy_slctd):
    if bool(hormone_therapy_slctd):
        temp_dataset = dataset[dataset['hormone_therapy'].isin(hormone_therapy_slctd)]
        temp_dataset.loc[temp_dataset['hormone_therapy'] == 0, 'hormone_therapy'] = "No"
        temp_dataset.loc[temp_dataset['hormone_therapy'] == 1, 'hormone_therapy'] = "Yes"
        return temp_dataset
    return dataset


# Función para aplicar el filtro por supervivencia
def apply_survival_filterYN(dataset, survival_slctd):
    if bool(survival_slctd):
        temp_dataset = dataset[dataset['overall_survival'].isin(survival_slctd)]
        temp_dataset.loc[temp_dataset['overall_survival'] == 0, 'overall_survival'] = "No"
        temp_dataset.loc[temp_dataset['overall_survival'] == 1, 'overall_survival'] = "Yes"
        return temp_dataset
    return dataset
    
# Función para aplicar el filtro por tipo de cancer
def apply_cancer_type(dataset, cancer_type_slctd):
    if bool (cancer_type_slctd):
        return dataset[dataset['cancer_type_detailed'].isin(cancer_type_slctd)]
    return dataset




#--------------------------------------------------------------------------------------------------------------------------------

# Aquí va el layout de la página con el grafico para supervivencia
layout_supervivencia = html.Div([
    html.Br(),
    html.Br(),
    html.H4("Treatments and Survival"),
    html.Br(),
    html.Br(),
    dbc.Row(
            [
                dbc.Col(html.Div([
                    dbc.Card(
                    dbc.CardBody(
                        [
                    html.H5('Filter Panel'),
                    html.Br(),
                        html.H5('Age at Diagnosis'), 
                    dcc.RangeSlider(id="agerange_filter2",
                                    marks={i: str(i) for i in range(20, 101, 10)},
                                    min=20,
                                    max=100,
                                    value=[22, 96]
                                ),
                            
                    html.Br(),
                        html.H5('Chemotherapy'), 
                    dcc.Checklist(id="chemotherapy_filterYN2",
                                  options=[
                                        {'label': ' Yes', 'value': 1},
                                        {'label': ' No', 'value': 0}
                                    ],
                                    value=[0, 1],
                                    labelStyle = dict(display='block')
                                ),
                   
                    html.Br(),
                        html.H5('Radiotherapy'), 
                    dcc.Checklist(id="Radiotherapy_filterYN2",
                                  options=[
                                        {'label': ' Yes', 'value': 1},
                                        {'label': ' No', 'value': 0}
                                    ],
                                    value=[0, 1],
                                    labelStyle = dict(display='block')
                                ),         
                            
                    html.Br(),
                        html.H5('Hormone Therapy'), 
                    dcc.Checklist(id="Hormonetherapy_filterYN2",
                                  options=[
                                        {'label': ' Yes', 'value': 1},
                                        {'label': ' No', 'value': 0}
                                    ],
                                    value=[0, 1],
                                    labelStyle = dict(display='block')
                                ),         
                                    
                   html.Br(),
                        html.H5('ER status'), 
                     dcc.Checklist(id="er_status_filterYN2",
                                   options=[
                                      {'label': ' Positive', 'value': 'Positive'},
                                      {'label': ' Negative', 'value': 'Negative'}
                                    ],
                                   value=['Positive', 'Negative'],
                                 labelStyle = dict(display='block')
                               ),
    
                      html.Br(),
                        html.H5('Breast surgery'), 
                    dcc.Checklist(id="type_of_breast_surgery_filterYN2",
                                  options=[
                                       {'label': ' Mastectomy', 'value': 'MASTECTOMY'},
                                       {'label': ' Breast Conserving', 'value': 'BREAST CONSERVING'}
                                    ],
                                    value=['MASTECTOMY', 'BREAST CONSERVING'],
                                  labelStyle = dict(display='block')
                                ),        
                            
                    html.Br(),
                            html.H5('Select Cancer Type'), 
                    dcc.Dropdown (id="cancer_type2",
                                  options=[
                                      {'label': i, 'value': i} for i in df.cancer_type_detailed.unique()],
                                  value=cancer_type_detailed,
                                  multi=True, 
                                  placeholder="Select Cancer Type"
                                 ),
                    
                ]))]), width=3),
                
                dbc.Col(html.Div([
                       
                    html.Br(),
                    
                    html.H5('Survival of Patients per Age at diagnosis'), 
                    dcc.Graph(id='barplot2'),
                    
                     html.Br(),
                    
                ])),
            ]
        )

], style={"marginLeft": "10px", "marginRight": "10px"})



#callback supervivencia


@app.callback(Output(component_id='barplot2', component_property='figure'),
              [Input(component_id='agerange_filter2', component_property='value'),
               Input(component_id='chemotherapy_filterYN2', component_property='value'),
               Input(component_id='Radiotherapy_filterYN2', component_property='value'),
               Input(component_id='Hormonetherapy_filterYN2', component_property='value'),
               Input(component_id='er_status_filterYN2', component_property='value'),
               Input(component_id='type_of_breast_surgery_filterYN2', component_property='value'),
               Input(component_id='cancer_type2', component_property='value')]
)


def update_graph (agerange_filter2,
                 chemotherapy_filterYN2,
                 Radiotherapy_filterYN2,
                 Hormonetherapy_filterYN2,
                 er_status_filterYN2,
                 type_of_breast_surgery_filterYN2,
                 cancer_type2):
    
    dfff = df.copy()
    # Aplicamos los filtros generales
    dfff = apply_agerange_filter2(dfff, agerange_filter2)
    dfff = apply_chemotherapy_filterYN2(dfff, chemotherapy_filterYN2)
    dfff = apply_Radiotherapy_filterYN2(dfff, Radiotherapy_filterYN2)
    dfff = apply_Hormonetherapy_filterYN2(dfff, Hormonetherapy_filterYN2)
    dfff = apply_er_status_filterYN2(dfff, er_status_filterYN2)
    dfff = apply_type_of_breast_surgery_filterYN2(dfff, type_of_breast_surgery_filterYN2)
    dfff = apply_cancer_type2(dfff, cancer_type2)

    
    #Se debe de retornar la cantidad de outputs que se tengan
    
    fig = px.density_contour(dfff, x="age_at_diagnosis", y="overall_survival_months",
                            labels= {"age_at_diagnosis":"Age at diagnosis", 
                                     "overall_survival_months":"Overall Survival (Months)"})
    fig.update_traces(contours_coloring="fill", contours_showlabels = True)
    
    return (fig)
    

# Función para aplicar el filtro por edad
def apply_agerange_filter2(dataset, age_slctd):
    if bool(age_slctd):
        return dataset[dataset['age_at_diagnosis'].isin(range(age_slctd[0], age_slctd[1] + 1))]
    return dataset

# Función para aplicar el filtro por quimioterapia
def apply_chemotherapy_filterYN2(dataset, chemotherapy_slctd):
    if bool(chemotherapy_slctd):
        temp_dataset = dataset[dataset['chemotherapy'].isin(chemotherapy_slctd)]
        temp_dataset.loc[temp_dataset['chemotherapy'] == 0, 'chemotherapy'] = "No"
        temp_dataset.loc[temp_dataset['chemotherapy'] == 1, 'chemotherapy'] = "Yes"
        return temp_dataset
    return dataset

# Función para aplicar el filtro por quimioterapia
def apply_Radiotherapy_filterYN2(dataset, radio_therapy_slctd):
    if bool(radio_therapy_slctd):
        temp_dataset = dataset[dataset['radio_therapy'].isin(radio_therapy_slctd)]
        temp_dataset.loc[temp_dataset['radio_therapy'] == 0, 'radio_therapy'] = "No"
        temp_dataset.loc[temp_dataset['radio_therapy'] == 1, 'radio_therapy'] = "Yes"
        return temp_dataset
    return dataset

# Función para aplicar el filtro por quimioterapia
def apply_Hormonetherapy_filterYN2(dataset, hormone_therapy_slctd):
    if bool(hormone_therapy_slctd):
        temp_dataset = dataset[dataset['hormone_therapy'].isin(hormone_therapy_slctd)]
        temp_dataset.loc[temp_dataset['hormone_therapy'] == 0, 'hormone_therapy'] = "No"
        temp_dataset.loc[temp_dataset['hormone_therapy'] == 1, 'hormone_therapy'] = "Yes"
        return temp_dataset
    return dataset

# Función para aplicar el filtro por tipo de cirugia de mama
def apply_type_of_breast_surgery_filterYN2(dataset, type_of_breast_surgery_slctd):
    if bool(type_of_breast_surgery_slctd):
        temp_dataset = dataset[dataset['type_of_breast_surgery'].isin(type_of_breast_surgery_slctd)]
        temp_dataset.loc[temp_dataset['type_of_breast_surgery'] == 'MASTECTOMY', 'type_of_breast_surgery'] = "MASTECTOMY"
        temp_dataset.loc[temp_dataset['type_of_breast_surgery'] == 'BREAST CONSERVING', 'type_of_breast_surgery'] = "BREAST CONSERVING"
        return temp_dataset
    return dataset

# Función para aplicar el filtro por er_status
def apply_er_status_filterYN2(dataset, er_status_slctd):
    if bool(er_status_slctd):
        temp_dataset = dataset[dataset['er_status'].isin(er_status_slctd)]
        temp_dataset.loc[temp_dataset['er_status'] == 'Negative', 'er_status'] = "Negative"
        temp_dataset.loc[temp_dataset['er_status'] == 'Positive', 'er_status'] = "Positive"
        return temp_dataset
    return dataset
    
# Función para aplicar el filtro por tipo de cancer
def apply_cancer_type2(dataset, cancer_type_slctd):
    if bool (cancer_type_slctd):
        return dataset[dataset['cancer_type_detailed'].isin(cancer_type_slctd)]
    return dataset



#-------------------------------------------------------------------------------------------------------------------------------


# El main layout es para ayudarnos con las dos pestañas
# El dcc.Location sirve para identificar en cual pestaña estamos actualmente
# El html.Div sirve para mostrar el contenido de cada pestaña (pestaña general o pestaña específica)
main_layout = html.Div([
    navbar,
    html.Br(),
    
    dbc.Tabs([
        dbc.Tab(layout_general, label="General Graphs"),
        dbc.Tab(layout_especifico, label="Specific Graphs"),
        dbc.Tab(layout_supervivencia, label="Survival Graph")
    ]),
    html.Footer(html.Img(src=app.get_asset_url("acib.png"), height="150px"))
])

# Iniciar el app.layout con el main_layout
app.layout = main_layout


# -----------------------------------------------------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False, port=5555)
    
    

Dash app running on http://127.0.0.1:5555/
